In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sklearn
# 正则表达式
import re
import sys
# 中心化
from sklearn.preprocessing import MinMaxScaler
# 标准化
from sklearn.preprocessing import StandardScaler
# 自定义的工具包
from my_tools import *

In [94]:
import warnings
warnings.filterwarnings("ignore")

### 读取数据

In [95]:
jibing = pd.read_excel("./某疾病数据.xlsx")

### 将汉字转换为数值

#### 性别

In [96]:
set(jibing.loc[:,"性别"])
dict_sex = {"女":0,"男":1}
for i in range(jibing.loc[:,"性别"].shape[0]):
    jibing.loc[:,"性别"][i] = dict_sex[jibing.loc[:,"性别"][i]]

#### 血型

In [97]:
set(jibing.loc[:,"血型"])

{'A', 'AB', 'B', 'O'}

In [98]:
dict_sex = {"A":0,"AB":1,"B":2,"O":3}
for i in range(jibing.loc[:,"血型"].shape[0]):
    jibing.loc[:,"血型"][i] = dict_sex[jibing.loc[:,"血型"][i]]
set(jibing.loc[:,"血型"])

{0, 1, 2, 3}

左右

In [99]:
set(jibing.loc[:,"左右"])

{'双侧', '右', '女', '左'}

查看各部分所占的比例，发现 左 和 右 所占的比例和接近 99%

对于比例较少的 女 和 双侧 可以认为是噪声，直接删除

In [100]:
length = jibing.loc[:,"左右"].shape[0]
length_nv = jibing[jibing["左右"] == "女"].shape[0]
length_sc = jibing[jibing["左右"] == "双侧"].shape[0]
length_z = jibing[jibing["左右"] == "左"].shape[0]
length_y = jibing[jibing["左右"] == "右"].shape[0]
print("左：" + str(length_z / length * 100) + "%")
print("右：" + str(length_y / length * 100) + "%")
print("双侧：" + str(length_sc / length * 100) + "%")
print("女：" + str(length_nv / length * 100) + "%")

左：32.76178812002449%
右：67.11573790569504%
双侧：0.0612369871402327%
女：0.0612369871402327%


删除某一行后使用 reset_index 保证索引的连续性

In [101]:
jibing = jibing.drop(labels=jibing[jibing['左右'] == "女"].index).reset_index(drop = True)
jibing = jibing.drop(labels=jibing[jibing['左右'] == "双侧"].index).reset_index(drop = True)

In [102]:
dict_lr = {"左":0,"右":1,}
for i in range(jibing.loc[:,"左右"].shape[0]):
    jibing.loc[:,"左右"][i] = dict_lr[jibing.loc[:,"左右"][i]]
set(jibing.loc[:,"左右"])

{0, 1}

#### 症状持续时间

In [103]:
set(jibing.loc[:,"症状持续时间"])

{'10余天',
 '10余年',
 '10天',
 '10年余',
 '10月',
 '10月   ',
 '10月余',
 '11天',
 '11月',
 '11月余',
 '12天',
 '14年',
 '15天',
 '15年',
 '17天',
 '17年',
 '18月',
 '1周',
 '1周余',
 '1天',
 '1天余',
 '1年',
 '1年余',
 '1年半',
 '1年半余',
 '1月',
 '1月余',
 '20余天',
 '20余年',
 '20天',
 '20天余',
 '20年余',
 '21天',
 '2周',
 '2周余',
 '2天',
 '2天余',
 '2年',
 '2年余',
 '2月',
 '2月余',
 '2月余 ',
 '3周余',
 '3周月',
 '3天',
 '3年',
 '3年余',
 '3月',
 '3月余',
 '3月余 ',
 '40余天',
 '4天',
 '4小时',
 '4年',
 '4年余',
 '4月',
 '4月余',
 '4月余   ',
 '50天',
 '5天',
 '5年',
 '5年余',
 '5月',
 '5月余',
 '6年余',
 '6月',
 '6月 ',
 '6月余',
 '6月余 ',
 '7天',
 '7年余',
 '7月余',
 '8年',
 '8年余',
 '8月',
 '8月   ',
 '8月余',
 '9年余',
 '9月',
 '9月余',
 '9月余 ',
 '半年',
 '半年余',
 '半月余',
 '数年',
 '无'}

In [104]:
for i in range(jibing.loc[:,"症状持续时间"].shape[0]):
    if re.search('小时', str(jibing.loc[:,"症状持续时间"][i])) is not None:
        jibing.loc[:,"症状持续时间"][i] = 0
    if re.search('天', str(jibing.loc[:,"症状持续时间"][i])) is not None:
        jibing.loc[:,"症状持续时间"][i] = 1
    if re.search('周', str(jibing.loc[:,"症状持续时间"][i])) is not None:
        jibing.loc[:,"症状持续时间"][i] = 2
    if re.search('月', str(jibing.loc[:,"症状持续时间"][i])) is not None:
        jibing.loc[:,"症状持续时间"][i] = 3
    if re.search('年', str(jibing.loc[:,"症状持续时间"][i])) is not None:
        jibing.loc[:,"症状持续时间"][i] = 4

In [105]:
set(jibing.loc[:,"症状持续时间"])

{0, 1, 2, 3, 4, '无'}

统计某一列各元素出现的个数

https://blog.csdn.net/zr1213159840/article/details/107818784?spm=1001.2101.3001.6650.3&utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-3-107818784-blog-103014532.pc_relevant_recovery_v2&depth_1-utm_source=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-3-107818784-blog-103014532.pc_relevant_recovery_v2&utm_relevant_index=6

In [106]:
jibing['症状持续时间'].value_counts()

4    797
3    756
1     58
2     18
无      1
0      1
Name: 症状持续时间, dtype: int64

无占很小的比例，直接删掉那一行


pandas 获取指定列中的某个值（范围）所属的行

https://blog.csdn.net/weixin_44222183/article/details/106187018

In [107]:
jibing[jibing['症状持续时间'] == "无"].index

Int64Index([1515], dtype='int64')

pandas 删除某一行

https://blog.csdn.net/LHJCSDNYL/article/details/124784943?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522166808474916800182765154%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=166808474916800182765154&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-124784943-null-null.142^v63^wechat,201^v3^add_ask,213^v2^t3_esquery_v1&utm_term=pandas%20%E5%88%A0%E9%99%A4%E6%9F%90%E4%B8%80%E8%A1%8C&spm=1018.2226.3001.4187

In [108]:
jibing = jibing.drop(labels=jibing[jibing['症状持续时间'] == "无"].index)
jibing = jibing.drop(labels=jibing[jibing['症状持续时间'] == 0 ].index)

In [109]:
jibing.head(5)

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型,结果
0,1,0,3,0.0,1,54,0,0,0,1,...,2.03,73.0,39.0,2.0,48.0,15.5,13.4,59.4,0,0
1,1,1,1,1.0,0,63,1,0,1,0,...,2.90,84.0,20.0,3.1,71.0,17.7,12.4,67.1,0,0
2,1,0,4,1.0,0,65,0,0,0,0,...,1.55,121.0,7.0,1.8,63.0,11.4,19.6,50.5,2,0
3,0,1,1,0.0,1,45,0,0,0,1,...,1.90,187.0,19.0,2.3,42.0,9.4,9.8,55.8,2,0
4,1,1,3,1.0,1,55,0,0,0,0,...,2.19,66.0,25.0,2.0,111.0,15.3,26.1,54.8,0,0


### 填补缺失值
未发现缺失值

In [110]:
jibing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1629 entries, 0 to 1630
Data columns (total 63 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   左右             1629 non-null   object 
 1   是否外伤           1629 non-null   int64  
 2   症状持续时间         1629 non-null   object 
 3   明显夜间痛          1629 non-null   float64
 4   性别             1629 non-null   object 
 5   年龄             1629 non-null   int64  
 6   高血压            1629 non-null   int64  
 7   高血脂            1629 non-null   int64  
 8   2型糖尿病          1629 non-null   int64  
 9   吸烟与否           1629 non-null   int64  
 10  饮酒与否           1629 non-null   int64  
 11  红细胞计数*10^12/L  1629 non-null   float64
 12  血红蛋白           1629 non-null   float64
 13  红细胞压积          1629 non-null   float64
 14  血小板计数          1629 non-null   float64
 15  血小板压积          1629 non-null   float64
 16  总蛋白g/L         1628 non-null   float64
 17  白蛋白g/L         1629 non-null   float64
 18  球蛋白g/L  

### 缺失值填补
缺失值占了很少的比例，所以直接去掉缺失值所在的行

In [111]:
jibing.dropna(axis=0,inplace=True)

In [112]:
jibing.head(5)

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型,结果
0,1,0,3,0.0,1,54,0,0,0,1,...,2.03,73.0,39.0,2.0,48.0,15.5,13.4,59.4,0,0
1,1,1,1,1.0,0,63,1,0,1,0,...,2.90,84.0,20.0,3.1,71.0,17.7,12.4,67.1,0,0
2,1,0,4,1.0,0,65,0,0,0,0,...,1.55,121.0,7.0,1.8,63.0,11.4,19.6,50.5,2,0
3,0,1,1,0.0,1,45,0,0,0,1,...,1.90,187.0,19.0,2.3,42.0,9.4,9.8,55.8,2,0
4,1,1,3,1.0,1,55,0,0,0,0,...,2.19,66.0,25.0,2.0,111.0,15.3,26.1,54.8,0,0


### 其他非法字符
小数点输入重复

多余的 + 

去量纲化的过程中发现有很多部分数字的小数点重复，
为了不对整体产生影响，决定将其删除

kmp 算法

ValueError: could not convert string to float: '22..9'

In [113]:
drop_index = []
for i in range(jibing.shape[0]):
    for j in range(jibing.shape[1]):
        p=".."
        p=list(p)
        s=list(str(jibing.iloc[i,j]))
        next=[0]
        Get_next(p, next)
        if kmp_match(s,p,next) != -1:
            drop_index.append(i)
        p = "+"
        p=list(p)
        s=list(str(jibing.iloc[i,j]))
        next=[0]
        Get_next(p, next)
        if kmp_match(s,p,next) != -1:
            drop_index.append(i)
drop_index

[155, 265, 356]

In [114]:
jibing = jibing.drop(labels=156,axis=0)
jibing = jibing.drop(labels=266,axis=0)
jibing = jibing.drop(labels=357,axis=0).reset_index()

## 打乱数据集

In [115]:
import random
id = [i for i in range(0,len(jibing))]
random.shuffle(id)
jibing_copy = jibing.copy()
for j in range(0,len(jibing)):
    jibing.iloc[j] = jibing_copy.iloc[id[j]]

In [116]:
jibing = jibing.iloc[:,1:]

In [117]:
jibing.head()

,左右,是否外伤,症状持续时间,明显夜间痛,性别,年龄,高血压,高血脂,2型糖尿病,吸烟与否,...,果糖胺,肌酸激酶,α-L-盐藻糖苷酶,乳酸,淀粉酶,同型半胱氨酸,铁,总铁结合力,血型,结果
0,1,0,3,0.0,1,51,0,0,0,1,...,1.97,62.0,18.0,1.4,80.0,23.0,14.2,51.6,0,0
1,1,0,4,0.0,0,69,0,0,0,0,...,1.71,72.0,20.0,2.5,103.0,10.3,6.5,57.1,2,0
2,1,0,4,0.0,0,30,0,0,0,0,...,1.67,32.0,17.0,1.3,52.0,10.7,25.3,64.1,0,0
3,1,0,4,1.0,1,59,0,0,0,1,...,2.05,132.0,16.0,1.6,55.0,21.2,35.6,71.3,0,0
4,1,0,4,1.0,1,52,0,0,0,1,...,1.72,81.0,22.0,1.2,81.0,14.8,20.1,61.4,3,0


### 必须重新读取一下 excel 否则会报错
- error :isnan

In [118]:
jibing.to_excel("./jibing_yuchuli_final.xlsx", index=False)

In [119]:
jibing = pd.read_excel("./jibing_yuchuli_final.xlsx")

## 使用 3σ 消除极值

In [120]:
jibing = three_sigema(jibing)

操作前有1624行
操作后有1598行
删去了26行


### 保存

In [121]:
jibing.to_excel("./jibing_yuchuli_final.xlsx", index=False)

In [122]:
jibing.shape

(1598, 63)